# MNIST Workshop

### Witajcie

Na tym workshopie zamiemy się rozpoznaniem odręczenie pisanych cyfr - klasycznym problemem w uczeniu maszynowym, który *wciąż* stosowany jest w wielu problemach jako podstawa do najnowszych badan naukowych. Warto poznać zbiór MNIST'a, ale przede wszystkim warto poznać techniki które pozwalają na pracę z danymi osadzonymi w przestrzeniach wielowymiarowych, takimi jak obrazy.

###  Dlaczego dane wielowymiarowe? 
Czym jest obraz gdy patrzymy na niego przez większość algorytów ML? To po prostu wektor - strzałka wskazujaca na punkt w przestarzeni $R^N$, gdzie $N$ oznacza ilość wymiarów w tej przestrzeni. Ile jest wymiarów? Tyle ile pikseli * tyle ile mamy kanałów w obrazie (przeważnie 3 - RGB). Brzmi to na początku abstrakcyjnie - ale taka reprezentacja obrazu faktycznie jest stosowana w uczeniu maszynowym! Zobaczmy w takim razie jak to wygląda na przykładzie MNIST - zbioru odręcznie pisanych cyfr, na monochromatycznych obrazkach o rozmiarach 28 na 28 pikseli.

## Jak wyglądają cyferki MNISTa?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.figsize"] = [16, 9]

In [ ]:
# show 2D matrices as tiles (takes 4D `examples` tensor with dims: rows x cols x tile_height x tile_width as input)
def tiles(examples):
    rows_count = examples.shape[0]
    cols_count = examples.shape[1]
    tile_height = examples.shape[2]
    tile_width = examples.shape[3]
    
    space_between_tiles = 2
    img_matrix = np.empty(shape=(rows_count * (tile_height + space_between_tiles) - space_between_tiles,  
                                 cols_count * (tile_width + space_between_tiles) - space_between_tiles))
    img_matrix.fill(np.nan)

    # TODO: fill in loops that copy 2D slices from 4D tensor into 2D grid to display
    
    return img_matrix

In [ ]:
import mnist
digits = np.reshape(mnist.train_images()[:12*24], newshape=(12, 24, 28, 28))

img = tiles(digits)
plt.matshow(img, cmap='gray', interpolation='none')
plt.axis('off')
plt.show()

In [ ]:
X = mnist.train_images().astype(np.float32) / 255.0
y = mnist.train_labels()
X.shape

In [ ]:
# reshape `X` so that the last two dimensions are collapsed into single dimension
# X = ???

# Wbudujmy nasze obserwacje ze zbioru MNIST w niskowymiarową przestrzeń 

Ciężko nam sobie wyobrazić jak wyglądają takie obserwacje w 784 wymiarach - dlatego powstały techniki redukcji wymiarowości, zachowujące jak najwięcej informacji o wzajemnym położeniu punktów w oryginalnej przestrzeni. Na początku zobaczymy jak działa klasyczna metoda (dość prosta, bo liniowa), polegająca na zmianie układu współrzędnych tak, by na kolejnych osiach zachowywać jak najwięcej wariancji z oryginalnego zbioru - **Principal Components Analysis**, czyli mówiąc krótko - **PCA**.

In [ ]:
def plot_2d_mnist_scatter(X, y):
    fig, plot = plt.subplots()
    fig.set_size_inches(16, 16)
    plt.prism()

    # TODO: plot each digit observations at given coordinates with seperate scatter and appropriate label
    
    plot.set_xticks(())
    plot.set_yticks(())

    plt.tight_layout()
    plt.legend()
    plt.show()

In [ ]:
SAMPLES_LIMIT = 2000
X_small = X[:SAMPLES_LIMIT]
y_small = y[:SAMPLES_LIMIT]

In [ ]:
# TODO: Use PCA function to embed `X_small` in two dimensions. Store the result in `X_pca_embedded`.

from sklearn.decomposition import PCA

# pca =  ???
# X_pca_embedded = ???

In [ ]:
X_pca_embedded.shape

In [ ]:
plot_2d_mnist_scatter(X_pca_embedded, y_small)

Ile wariancji udało się zachować w naszych dwóch pierwszych składowych głównych? 

In [ ]:
pca.explained_variance_ratio_

Jak widać, rzut z 784 wymiarów zachował zadziwiająco dużo informacji, utrzymując poszczególne obserwacje we względnie zwartych grupach. W dwóch pierwszych składowych głównych udało nam się zachować kolejno 10.0% i 7.4% wariancji z oryginalnych danych - dużo straciliśmy, ale zachowanie 17.4% wariancji przy pozostawieniu tylko 0.26% ze wszystkich wymiarów to i tak duże osiągnięcie. 

Czy możemy lepiej? Oczywiście! Musimy jednak skorzystać z modelu, który będzie w stanie zamodelować nieliniowe relacje w danych - znajdziemy tzw. dwuwymiarowy manifold na którym układają sie dane w przestrzeni 784 wymiarowej stosując technikę **t-distributed Stochastic Neighbour Embedding**, znaną pod skrótem **t-SNE**.

In [ ]:
# TODO: Use t-SNE from sklearn package to embed `X_small` in two dimensions. Store the result in `X_tsne_embedded`.

from sklearn.manifold import TSNE

# tsne = ???
# X_tsne_embedded = ???

In [ ]:
X_tsne_embedded.shape

In [ ]:
plot_2d_mnist_scatter(X_tsne_embedded, y_small)

Jeśli mamy zainstalowane plotly, spróbujmy z interaktywnym wykresem w 3 wymiarach.

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

def plot_3d_mnist_plotly_scatter(X, y):
    def make_trace(i):
        digit_indeces = y == i
        return go.Scatter3d(
            x=X[digit_indeces, 0],
            y=X[digit_indeces, 1],
            z=X[digit_indeces, 2],
            mode='markers',
            name=str(i),
            marker=dict(
                color=i,
                colorscale='Jet',
                size=4,
                symbol='circle',
                line=dict(
                    color='rgb(204, 204, 204)',
                    width=1)))
        
    traces = [make_trace(i) for i in range(10)]
    
    layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0))
    fig = go.Figure(data=traces, layout=layout)
    iplot(fig)

In [ ]:
# TODO: Use t-SNE from sklearn package to embed `X_small` in three dimensions. Store the result in `X_tsne_3d_embedded`.

# tsne_3d = ??? 
# X_tsne_3d_embedded = ???

In [ ]:
X_tsne_3d_embedded.shape

In [ ]:
plot_3d_mnist_plotly_scatter(X_tsne_3d_embedded, y_small)

# Spróbujmy sklasyfikować obserwacje ze zbioru MNIST

Jak widać, zaawansowane techniki redukcji wymiarowości pozwalają na wyseparowanie wyraźnych klastrów naszych obserwacji z 784 wymiarowej przestrzeni. Skoro tak, nie powinniśmy mieć większego problemu z wytrenowaniem klasyfikatora, który poradzi sobie z zadaniem rozpoznawiania cyferek na obrazkach. 

Do tego celu skorzystamy z klasyfikatora opartego o **Support Vector Machines (SVM)**, który choć jest modelem liniowym, potrafi sobie poradzić ze znalezieniem złożonego manifoldu na którym leżą obserwacje przez tzw. **kernel trick** - czyli transformacje przestrzeni w taki sposób, aby obserwacje były separowalne liniowo. My skorzystamy z tak zwanego **Gaussian Kernel'a**, znanego inaczej **Radial Basis Function**.

In [ ]:
from sklearn.model_selection import train_test_split
SAMPLES_LIMIT=10000
X_train, X_test, y_train, y_test = train_test_split(X[:SAMPLES_LIMIT], y[:SAMPLES_LIMIT], test_size=0.2)

In [ ]:
from sklearn import svm
classifier = svm.SVC(C=1, gamma=0.001)
classifier.fit(X_train, y_train)

In [ ]:
from sklearn import metrics

predicted = classifier.predict(X_test)

print("Classification report for classifier {}:\n{}\n".format(
    classifier, metrics.classification_report(y_test, predicted)))

W macierzy pomyłek możemy zauważyć które klasy są mylone z którymi. Widzimy, że niektóre klasy mylą się z innymi znacznie częściej od pozostałych. Czy jest w tym jakaś zależność? Czy stoi za tym jakaś reguła? Jak to się ma do naszych rzutów w przestrzeń niskowymiarową za pomocą t-SNE?

Zobaczmy! 

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, predicted)
print("Confusion matrix:\n{}".format(confusion_matrix))

# A może jakieś proste podejście z sieciami konwolucyjnymi?

Nasze podejście gubi dość istotną informację o modalności na której pracujemy! Obraz ma dość specyficzną strukturę - piksele obok siebie są skorelowane i podobne wzorce występują w różnych fragmentach obrazu. Możemy skorzystać z tej wiedzy - jeżeli zbudujemy model tak, aby uwzględniał to co wiem *a-priori* o naszych danych, powinien sobie poradzić lepiej w zadaniu klasyfikacji. 

Dokładnie to robią **sieci konwolucyjne**, znane też jako **CNN's (convolutional neural nets)**!

In [ ]:
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)
sess = tf.InteractiveSession()

In [ ]:
x_placeholder = tf.placeholder(tf.float32, shape=[None, 784])
y_placeholder = tf.placeholder(tf.int32, shape=[None])
y_onehot = tf.one_hot(y_placeholder, 10, 1.0, 0.0)

x_image = tf.reshape(x_placeholder, [-1,28,28,1])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# first layer convolutional filters will have 5x5x1 dimensions, and theres 32 of them
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# first convnet layer (takes [?, 28, 28, 1] tensor as input, output [?, 14, 14, 32] due to 2x2 pooling)
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [ ]:
# TODO: define parameter variables so that we have a correct filter tensor

# second layer convolutional filters will have 5x5x32 dimensions (notice depth increase!) - there should be 64 filters
# W_conv2 = weight_variable([5, 5, 32, 64])
# b_conv2 = bias_variable([64])

# second convnet layer (takes [?, 28, 28, 1] tensor as input, output [?, 14, 14, 32] due to 2x2 pooling)
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [ ]:
# then we apply first fully connected layer at the output
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# then some dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# and finally layer that computes logits for softmax layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

logits = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y_onehot, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_onehot, logits=logits))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy_loss)

init = tf.global_variables_initializer()

In [ ]:
sess.run(init)

In [ ]:
from itertools import cycle

def chunks(seq, size):
    return [seq[pos:pos + size] for pos in range(0, len(seq), size)]

def cycle_chunks(seq, size):
    return cycle(chunks(seq, size))

In [ ]:
BATCH_SIZE = 128
BATCHES_COUNT = 500
for i, (batch, labels) in zip(range(BATCHES_COUNT), 
                              zip(cycle_chunks(X_train, BATCH_SIZE), 
                                  cycle_chunks(y_train, BATCH_SIZE))):
    if i % 10 == 0:
        train_accuracy = accuracy.eval(feed_dict={x_placeholder:batch, y_placeholder: labels, keep_prob: 1.0})
        print("Step {0}, training accuracy {1:>2.2f}".format(i, 100 * train_accuracy))
    train_step.run(feed_dict={x_placeholder: batch, y_placeholder: labels, keep_prob: 0.5})

print("Test accuracy {0:>2.2f}".format(
    100 * accuracy.eval(feed_dict={x_placeholder: X_test, y_placeholder: y_test, keep_prob: 1.0})))

Bazowaliśmy na części zbioru MNIST - dość łatwo możemy zwiększyć *accuracy* do poziomy około 98%. W praktyce najnowsze metody znacznie przekraczają poziom 99%. Najważnieszy wniosek jest taki, że dość łatwo udało nam się wykorzystać informację, którą znaliśmy *a-priori*, o strukturze danych do sformułowania rozwiązania dającego lepsze rezultaty. No cóż, pewnie dlatego w uczeniu maszynowym mamy tak wiele różnych modeli - po prostu każda modalność rządzi się swoimi prawami.

## Dzięki za udział w workshopie, do zobaczenia na hackatonie!